In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [ ]:
# To store the data in the "out" folder
import os

os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [ ]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um", material='silicon')
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.NBR_1)
    d.add_transmon_plus_resonator(design, group=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.NBR_1)
    d.add_chargeline(design, group=n.NBR_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

In [ ]:
MINI_STUDY_GROUP = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

In [ ]:
group_runs = 2
group_passes_cap = 4

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
close_ansys()